# Major Model Analysis Ahead

    In this Notebook, we compare the performance of different models we trained. We use MSE and a normalized pearson correlation as performance metrics.

In [1]:
import sys
sys.path.append('../')
sys.path.append('../../')

In [2]:
from utils.model_analysis import * 
import regex as re
import pandas as pd
import torch.nn.functional as F

# Predicting and Evaluating Different Models

In [3]:
comparison = {}
models = []
corr_scores = []
mse_scores = []

## Alexnet - Complex Head - No PCA

In [4]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/alexnet_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_19:27:03.pt']
                   
backbone = 'alexnet'
subject_ids = list(range(1,8+1))

In [6]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'NO_PCA'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

  0%|          | 0/39198 [00:00<?, ?it/s]/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
100%|██████████| 39198/39198 [00:01<00:00, 31637.31it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear

100%|██████████| 39198/39198 [00:01<00:00, 26634.92it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear

100%|██████████| 39198/39198 [00:01<00:00, 32684.28it/s]


Subject 8 done.
Model alexnet_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_19:27:03.ptNO_PCA done.


## Alexnet - Simple Head - PCA 100

In [7]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_20:37:28.pt'         ]
                   
backbone = 'alexnet'
subject_ids = list(range(1,8+1))

In [8]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + '_SIMPLE_HEAD'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)

        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=True).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        
        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)
        
        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=921

100%|██████████| 39548/39548 [00:19<00:00, 2029.99it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear

100%|██████████| 39548/39548 [00:19<00:00, 2045.12it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear

100%|██████████| 39511/39511 [00:19<00:00, 2026.88it/s]

Subject 8 done.
Model 03-01-24/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_20:37:28.pt_SIMPLE_HEAD done.


## Resnet 18 - Complex Head - PCA 100

In [9]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet18_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_128_TIME_2024-01-02_16:34:13.pt']
                   
backbone = 'resnet18'
subject_ids = list(range(1,8+1))

In [10]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)

        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        
        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)
        
        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

  0%|          | 0/39548 [00:00<?, ?it/s]/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
100%|██████████| 39548/39548 [00:01<00:00, 33129.99it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39548/39548 [00:01<00:00, 35334.38it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39548/39548 [00:01<00:00, 24790.51it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39511/39511 [00:01<00:00, 33346.28it/s]


Subject 8 done.
Model 03-01-24/resnet18_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_128_TIME_2024-01-02_16:34:13.pt done.


##  Resnet 18 - Simple Head - No PCA

In [11]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet18_LR1e-05_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_19:03:14.pt']
                   
backbone = 'resnet18'
subject_ids = list(range(1,8+1))

In [12]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'SIMPLE_HEAD_NO_PCA'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics  per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = True).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    #models.append(model_name)
    #corr_scores.append(np.mean(corr_list))
    #mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

100%|██████████| 39198/39198 [00:19<00:00, 2020.24it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:19<00:00, 2035.75it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:19<00:00, 2041.27it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:19<00:00, 2044.67it/s]

Subject 8 done.
Model resnet18_LR1e-05_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_19:03:14.ptSIMPLE_HEAD_NO_PCA done.


## Resnet 34 - Complex Head - No PCA

In [13]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet34_LR1e-05_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_21:25:08.pt']
backbone = 'resnet34'
subject_ids = list(range(1,8+1))

In [14]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'NO_PCA'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

100%|██████████| 39198/39198 [00:01<00:00, 36634.97it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:01<00:00, 32048.38it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:01<00:00, 32889.22it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

100%|██████████| 39198/39198 [00:01<00:00, 32688.58it/s]


Subject 8 done.
Model resnet34_LR1e-05_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_21:25:08.ptNO_PCA done.


## Resnet 101 - Complex Head - PCA 100

In [15]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet101_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_17:15:40.pt']                   
backbone = 'resnet101'
subject_ids = list(range(1,8+1))

In [17]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics  per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)

        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        
        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)
        
        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 25

100%|██████████| 39548/39548 [00:01<00:00, 33495.06it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39548/39548 [00:01<00:00, 35006.91it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39548/39548 [00:01<00:00, 31737.06it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39511/39511 [00:01<00:00, 34602.19it/s]


Subject 8 done.
Model 03-01-24/resnet101_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_17:15:40.pt done.


## Resnet101 - Complex Head - No PCA

In [18]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet101_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_23:24:48.pt']               
backbone = 'resnet101'
subject_ids = list(range(1,8+1))

In [19]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'NO_PCA'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list)) 
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 25

100%|██████████| 39198/39198 [00:01<00:00, 32265.16it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:01<00:00, 29410.05it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:01<00:00, 31735.88it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:01<00:00, 31849.39it/s]

Subject 8 done.
Model resnet101_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_23:24:48.ptNO_PCA done.


## Resnet 152 - Complex Head - PCA 100

In [20]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet152_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_15:16:04.pt',
         '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_14:57:27.pt'
         ]
                   
backbone = 'resnet152'
subject_ids = list(range(1,8+1))

In [21]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)

        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=False).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        
        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)
        
        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 25

100%|██████████| 39548/39548 [00:01<00:00, 34220.19it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39548/39548 [00:01<00:00, 32543.95it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39548/39548 [00:00<00:00, 49669.05it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39511/39511 [00:01<00:00, 34056.46it/s]


Subject 8 done.
Model 03-01-24/resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_14:57:27.pt done.


## ResNet152 - Simple Head - No PCA

In [22]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/03-01-24/resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-03_01:23:21.pt']
                   
backbone = 'resnet152'
subject_ids = list(range(1,8+1))

In [23]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'03-01-24/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'SIMPLEA_HEAD_NO_PCA'
    corr_list = []
    mse_list = []
    #loop over all subjects to calculate performance metrics per subject
    for subject_id in subject_ids:
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
            
        activation = torch.Tensor(brain[:,:39198])

        output_size = activation.shape[1]

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('../utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head = True).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            preds[i,:] = pred.detach().numpy()

        predictions = torch.Tensor(preds)

        #calculate normalized corr score
        corr = norm_pearson(gt_activation=activation, pred_activation=predictions)

        mse_score= F.mse_loss(predictions, activation)
        #append to list
        corr_list.append(corr)
        mse_list.append(mse_score)

        print(f'Subject {subject_id} done.')

    models.append(model_name)
    corr_scores.append(np.mean(corr_list))
    mse_scores.append(np.mean(mse_list))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 25

100%|██████████| 39198/39198 [00:20<00:00, 1882.30it/s]


Subject 1 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:19<00:00, 2030.51it/s]


Subject 2 done.
Shape of pca_brain:  (908, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

...

Subject 6 done.
Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:19<00:00, 2003.79it/s]


Subject 7 done.
Shape of pca_brain:  (877, 39511)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0

100%|██████████| 39198/39198 [00:19<00:00, 2054.54it/s]

Subject 8 done.
Model resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-03_01:23:21.ptSIMPLEA_HEAD_NO_PCA done.


# Results

In [24]:
comparison = pd.DataFrame({'Model': models, 'Normalized Pearson Score': corr_scores, 'MSE Score': mse_scores})
pd.set_option('max_colwidth', 130)
comparison.sort_values('Normalized Pearson Score', ascending=False)

,Model,Normalized Pearson Score,MSE Score
1,03-01-24/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_20:37:28.pt_SIMPLE_HEAD,0.501906,0.903682
8,resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-03_01:23:21.ptSIMPLEA_HEAD_NO_PCA,0.499481,0.539091
0,alexnet_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_19:27:03.ptNO_PCA,NaN,0.539090
2,03-01-24/resnet18_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_128_TIME_2024-01-02_16:34:13.pt,NaN,0.539199
3,resnet34_LR1e-05_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_21:25:08.ptNO_PCA,NaN,0.539090
4,03-01-24/resnet101_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_17:15:40.pt,NaN,0.539199
5,resnet101_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_23:24:48.ptNO_PCA,NaN,0.539089
6,03-01-24/resnet152_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_15:16:04.pt,NaN,0.539199
7,03-01-24/resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_14:57:27.pt,NaN,0.539194


In [25]:
comparison.sort_values('MSE Score', ascending=True)

,Model,Normalized Pearson Score,MSE Score
5,resnet101_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_23:24:48.ptNO_PCA,NaN,0.539089
0,alexnet_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_19:27:03.ptNO_PCA,NaN,0.539090
3,resnet34_LR1e-05_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_32_TIME_2024-01-02_21:25:08.ptNO_PCA,NaN,0.539090
8,resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-03_01:23:21.ptSIMPLEA_HEAD_NO_PCA,0.499481,0.539091
7,03-01-24/resnet152_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_14:57:27.pt,NaN,0.539194
4,03-01-24/resnet101_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_17:15:40.pt,NaN,0.539199
2,03-01-24/resnet18_LR0.00015_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_128_TIME_2024-01-02_16:34:13.pt,NaN,0.539199
6,03-01-24/resnet152_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_15:16:04.pt,NaN,0.539199
1,03-01-24/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_True_EPOCHS50_BATCHSIZE_64_TIME_2024-01-02_20:37:28.pt_SIMPLE_HEAD,0.501906,0.903682


In [26]:
comparison.to_csv('comparison_results.csv')